In [9]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import json
from types import SimpleNamespace
import sys
from torch.utils.data import DataLoader, TensorDataset, Dataset
import tensorflow as tf

class CustomDataset(Dataset):
    def __init__(self, data, col, seq_len):
        self.data = data
        self.seq_len = seq_len
        self.col = col

    def __len__(self):
        return len(self.data) - self.seq_len + 1

    def __getitem__(self, idx):
        sample = self.data[idx : idx + self.seq_len]
        return sample[:, : self.col], sample[-1, self.col :].unsqueeze(0), sample[-1, : self.col].unsqueeze(0)

class PrepareData:
    def __init__(self, config):
        self.config = config

    def load_data(self, num_of_houses):
        # read from train list and concatenate the data
        for i in range(len(num_of_houses)):
            data = pd.read_csv(self.config.data_path + str(num_of_houses[i]) + '_compressed.csv')
            if i == 0:
                all_data = data
            else:
                all_data = pd.concat([all_data, data], axis=0)
        return all_data
    
    # write a function using MinMax scalar to normalize the data
    def normalize_data(self, data, normalize = True, scaler = None):
        if normalize:
            #z score normalization
            if scaler == None:
                scaler = StandardScaler()
                data = scaler.fit_transform(data)
                print("1")
            else:
                data = scaler.transform(data)
                print("2")
        else:
            scaler = None
            # data = scaler.fit_transform(data)
            data = data.to_numpy()
            # scaler = 0
        return data, scaler
    
    def segment_ev_load_data(self, data, window_length, num_of_houses, col, single_output = False, model_type = 'regression'):
        
        data = torch.from_numpy(data).to(self.config.device).float()
        segmented_data = []
        ground_truth = []
        for i in range(len(num_of_houses)):
            per_house_data = data[data[:, 0] == num_of_houses[i]]
            # Calculate the number of segments
            num_segments = (len(per_house_data) - window_length) + 1
            # Perform sliding window segmentation
            for start in range(num_segments):
                segmented_data.append(per_house_data[start:start + window_length, 1:col])
                if single_output:
                    ground_truth.append(per_house_data[start + window_length - 1, col:])
                else:
                    ground_truth.append(per_house_data[start:start + window_length, col:])

        # Perform sliding window segmentation
        # for start in range(num_segments):
        #     segmented_data = torch.cat(segmented_data, data[start:start + window_length, 1:])
        #     ground_truth = torch.cat(ground_truth, data[start:start + window_length, 0:1])

        # segmented_3d = segmented_data.reshape(segmented_data.shape[0], segmented_data.shape[1], -1)
        # ground_truth = ground_truth.reshape(ground_truth.shape[0], ground_truth.shape[1], -1)

        segmented_data = torch.stack(segmented_data)
        ground_truth = torch.stack(ground_truth)

        segmented_data = segmented_data[: -(segmented_data.shape[0] % self.config.batch_size)]
        ground_truth = ground_truth[: -(ground_truth.shape[0] % self.config.batch_size)]
        return segmented_data, ground_truth
        
    #write a function for train test loader
    def get_data_loader(self, data_preprocess, columns, seen = False, model_type = 'regression'):
        data = self.load_data(self.config.train)
        data = data_preprocess.preprocess_data(data)

        if seen:
            train_data, test_data = self.split_train_test(data)
            # return train_data, train_data, test_data, test_data, None, None
            id_col_train = train_data['dataid'].to_numpy().reshape(-1, 1)
            id_col_test = test_data['dataid'].to_numpy().reshape(-1, 1)
            train_data, scaler_train = self.normalize_data(train_data[columns], True)
            test_data, scalar_test = self.normalize_data(test_data[columns], True, scaler_train)
            train_data = np.concatenate((id_col_train, train_data), axis=1)
            test_data = np.concatenate((id_col_test, test_data), axis=1) 
            train_data, train_ground_truth = self.segment_ev_load_data(train_data, self.config.lag_size, self.config.train, self.config.col, self.config.single_output, model_type)
            test_data, test_ground_truth = self.segment_ev_load_data(test_data, self.config.lag_size, self.config.test, self.config.col, self.config.single_output, model_type)

            if model_type == 'classification':
                train_ground_truth = train_ground_truth.unsqueeze(1)
                test_ground_truth = test_ground_truth.unsqueeze(1)
        
        else:
            id_col = data['dataid'].to_numpy().reshape(-1, 1)
            train_data, scaler_train = self.normalize_data(data[columns], False)
            train_data = np.concatenate((id_col, train_data), axis=1)
            train_data, train_ground_truth = self.segment_ev_load_data(train_data, self.config.lag_size, self.config.train, self.config.col, self.config.single_output, model_type)

            data = self.load_data(self.config.test)
            data = data_preprocess.preprocess_data(data)
            id_col = data['dataid'].to_numpy().reshape(-1, 1)
            test_data, scalar_test = self.normalize_data(data[columns], False)
            test_data = np.concatenate((id_col, test_data), axis=1)
            test_data, test_ground_truth = self.segment_ev_load_data(test_data, self.config.lag_size, self.config.test, self.config.col, self.config.single_output, model_type)
        # test_data, test_ground_truth = self.segment_ev_load_data(test_data, self.config.lag_size)

        # train_dataset = CustomDataset(train_data, train_ground_truth)
        # test_dataset = CustomDataset(test_data, test_ground_truth)

        # train_loader = DataLoader(train_dataset, batch_size=self.config.batch_size, shuffle=False)
        # test_loader = DataLoader(test_dataset, batch_size=self.config.batch_size, shuffle=False)

        return train_data, train_ground_truth, test_data, test_ground_truth, scaler_train, scalar_test
        # return train_loader, test_loader, scaler_train, scalar_test
    
    def split_train_test(self, data):
        train_data = data[data['local_15min'] < self.config.split_date]
        test_data = data[data['local_15min'] >= self.config.split_date]
        return train_data, test_data
    
    # write a function using dataloader function
    def get_data_loader_unet(self, data_preprocess, columns):
        data = self.load_data(self.config.train)
        data = data_preprocess.preprocess_data(data)

        train_data, test_data = self.split_train_test(data)

        train_data, scaler_train = self.normalize_data(train_data[columns], True)
        test_data, scalar_test = self.normalize_data(test_data[columns], True, scaler_train)

        train_data = torch.tensor(train_data, dtype=torch.float32).to(self.config.device)
        test_data = torch.tensor(test_data, dtype=torch.float32).to(self.config.device)

        train_dataset = CustomDataset(train_data, self.config.col, self.config.lag_size)
        train_dataloader = DataLoader(train_dataset, batch_size=self.config.batch_size, shuffle=False, drop_last=True)

        test_dataset = CustomDataset(test_data, self.config.col, self.config.lag_size)
        test_dataloader = DataLoader(test_dataset, batch_size=self.config.batch_size, shuffle=False, drop_last=True)

        return train_dataloader, test_dataloader, scaler_train, scalar_test
    
class DatasetForQuantile(torch.utils.data.Dataset):
    def __init__(self,  inputs, targets, model_type, seq_len=99, target_index=None):
        self.inputs = inputs
        self.targets = targets
        self.model_type = model_type
        seq_len = seq_len 
        self.seq_len = seq_len
        self.len = self.inputs.shape[0] - self.seq_len
        self.target_index = target_index
        self.indices = np.arange(self.inputs.shape[0])
    def __len__(self):
        'Denotes the total number of samples'
        return self.len
    
    def get_sample(self, index):
        indices = self.indices[index : index + self.seq_len]
        inds_inputs=sorted(indices[:self.seq_len])
        if self.model_type == 'seq2seq':
            inds_targs=sorted(indices[:self.seq_len])
        else:
            if self.target_index is not None:
                inds_targs = sorted(indices[self.target_index - 1:self.target_index])
            else:
                inds_targs=sorted(indices[self.seq_len-1:self.seq_len])

        return self.inputs[inds_inputs], self.targets[inds_targs]

    def __getitem__(self, index):
        inputs, target = self.get_sample(index)
        # check if tensor of input is 2d
        if len(inputs.shape) == 2:
            inputs = torch.tensor(inputs).float()
        else:
            inputs = torch.tensor(inputs).unsqueeze(-1).float()
        
        return inputs, torch.tensor(target).float().squeeze()

class PrepareDataForQuantile:
    def __init__(self, config=None):
        self.config = config

    def normalize_data(self, data, type = 'minmax', normalize = True, scaler = None):
        if normalize:
            #z score normalization
            if scaler == None:
                if type == 'minmax':
                    scaler = MinMaxScaler()
                else:
                    scaler = StandardScaler()
                data = scaler.fit_transform(data)
            else:
                data = scaler.transform(data)
        else:
            scaler = None
            # data = scaler.fit_transform(data)
            data = data.to_numpy()
            # scaler = 0
        return data, scaler
    
    def spilit_data(self, data):
        split_1 = int(0.60 * len(data))
        split_2 = int(0.70 * len(data))
        train = data[:split_1]
        validation = data[split_1:split_2]
        test = data[split_2:]
        return train, validation, test
    
    def prepare_dataloader(self, x, y, seq_len, target_index=None):
        x_train, x_val, x_test = self.spilit_data(x)
        y_train, y_val, y_test = self.spilit_data(y)

        train_dataloader = DataLoader(DatasetForQuantile(x_train, y_train, self.config.model_type, seq_len, target_index), batch_size=self.config.batch_size, shuffle=False, drop_last=True, pin_memory=True)
        test_dataloader = DataLoader(DatasetForQuantile(x_test, y_test, self.config.model_type, seq_len, target_index), batch_size=self.config.batch_size, shuffle=False, drop_last=True, pin_memory=True)
        val_dataloader = DataLoader(DatasetForQuantile(x_val, y_val, self.config.model_type, seq_len, target_index), batch_size=self.config.batch_size, shuffle=False, drop_last=True, pin_memory=True)

        return train_dataloader, test_dataloader, val_dataloader
    
    def get_data_loaders(self, start_col, end_col, seq_len):
        #read numpy data
        x, y = self.load_data()
        y = self.get_specific_col_data(y, start_col, end_col)

        train_dataloader, test_dataloader, val_dataloader = self.prepare_dataloader(x, y, seq_len)

        #unseen test data
        x_unseen, y_unseen = self.load_data_unseen()
        y_unseen = self.get_specific_col_data(y_unseen, start_col, end_col)
        _, test_dataloader, val_dataloader = self.prepare_dataloader(x_unseen, y_unseen, seq_len)

        return train_dataloader, test_dataloader, val_dataloader
    
    def load_data(self):
        x = np.load("Dataset/pecan_street/austin/npy_dataset/8156_input_with_ev_unet.npy")
        y = np.load("Dataset/pecan_street/austin/npy_dataset/8156_target_with_ev_unet.npy")
        return x, y
    
    def load_data_unseen(self):
        x = np.load("Dataset/pecan_street/austin/npy_dataset/661_input_with_ev_unet.npy")
        y = np.load("Dataset/pecan_street/austin/npy_dataset/661_target_with_ev_unet.npy")
        return x, y
    
    def get_specific_col_data(self, y, start_col, end_col):
        y = y[:, start_col:end_col]
        return y
    
    def get_data_for_cnn(self, data, input_cols, output_cols, seq_len):
        x = self.get_input_features(data[input_cols])
        y = data[output_cols]
        x, scalar_x = self.normalize_data(x, type="sd")
        y, scalar_y = self.normalize_data(y, type="sd")

        train_dataloader, test_dataloader, val_dataloader = self.prepare_dataloader(x, y, seq_len, self.config.output_col_num)
        return train_dataloader, test_dataloader, val_dataloader, scalar_x, scalar_y
        
    def get_input_features(self, data):
        data['local_15min'] = pd.to_datetime(data['local_15min'], utc=True)
        data['hour'] = data['local_15min'].dt.hour
        data['minute'] = data['local_15min'].dt.minute
        hour_of_day_tensor = torch.tensor(data['hour'].values, dtype=torch.float32)
        data['sine_h'] = torch.sin(2 * torch.pi * hour_of_day_tensor / 24)
        data['cos_h'] = torch.cos(2 * torch.pi * hour_of_day_tensor / 24)
        minute_of_day_tensor = torch.tensor(data['minute'].values, dtype=torch.float32)
        data['sine_m'] = torch.sin(2 * torch.pi * minute_of_day_tensor / 60)
        data['cos_m'] = torch.cos(2 * torch.pi * minute_of_day_tensor / 60)
        data.drop(['local_15min', 'hour', 'minute'], axis=1, inplace=True)
        return data
    
    def get_data_for_vae(self, start_col, end_col, width, strides, test_from=0, set_type="both"):

        def seq_dataset(x, y, width, stride):
            x_ = []
            y_ = []

            for t in range(0, x.shape[0]-width, stride):
                x_.append(x[t:t+width])
                y_.append(y[t:t+width])

            x_ = np.array(x_).reshape([-1, width, 1])
            y_ = np.array(y_).reshape([-1, width, 1])

            return x_, y_

        def select_ratio(x, y, ratio, set_type, test_from=0):
            num_data = x.shape[0]
            if set_type == "train":
                ind = np.random.permutation(num_data)
            else:
                ind = np.arange(num_data)
            
            min_data = int(num_data*test_from)
            max_data = int(num_data*ratio)

            if ratio == 0:
                return x[ind[0:1]], y[ind[0:1]]
            else:
                return x[ind[:max_data]], y[ind[:max_data]]
            
        def split_train_test(x, y, ratio):
            # num_data = x.shape[0]
            # ind = np.random.permutation(num_data)
            # min_data = int(num_data*ratio)
            # return x[ind[:min_data]], y[ind[:min_data]], x[ind[min_data:]], y[ind[min_data:]]
            split_1 = int(ratio * len(x))
            x_train = x[:split_1]
            x_test = x[split_1:]
            y_train = y[:split_1]
            y_test = y[split_1:]
            
            return x_train, y_train, x_test, y_test

        def create_dataset(width, strides, set_type, test_from=0):

            x, y = self.load_data() # Load complete dataset
            y = self.get_specific_col_data(y, start_col, end_col) # Select the appliance
            x_train, y_train, _, _ = split_train_test(x, y, 0.7) # Split the dataset in train and test

            x_unseen, y_unseen = self.load_data_unseen() # Load unseen dataset
            y_unseen = self.get_specific_col_data(y_unseen, start_col, end_col) # Select the appliance
            _, _, x_test, y_test = split_train_test(x_unseen, y_unseen, 0.7) # Split the unseen dataset in train and test

            print("x_train shape : ", x_train.shape)
            print("y_train shape : ", y_train.shape)
            print("x_test shape : ", x_test.shape)
            print("y_test shape : ", y_test.shape)
            x_train, y_train = seq_dataset(x_train, y_train, width, strides) # Divide dataset in window
            x_test, y_test = seq_dataset(x_test, y_test, width, strides) # Divide dataset in window
            # for h, r in zip(dataset[set_type]["house"], dataset[set_type]["ratio"]):
                
            #     if set_type == "test":
            #         x_r, y_r = select_ratio(x_, y_, r, set_type, test_from=test_from)
            #     else:
            #         x_r, y_r = select_ratio(x_, y_, r, set_type)# Select the proportion needed

            #     print("Total house {} : x:{}, y:{}".format(h, x_.shape, y_.shape))
            #     print("Ratio house {} : {}, x:{}, y:{}".format(h, r, x_r.shape, y_r.shape))

            #     x_tot = np.vstack([x_tot, x_r])
            #     y_tot = np.vstack([y_tot, y_r])

            # print("Complete dataset : x:{}, y{}".format(x_tot.shape, y_tot.shape))

            return x_train, y_train, x_test, y_test

        ###############################################################################
        # Load dataset
        ###############################################################################
        width = width
        stride = strides

        x_train, y_train, x_test, y_test = create_dataset(width, stride, set_type, test_from=test_from)

        return x_train, y_train, x_test, y_test
        
class QuantileLoss(torch.nn.Module):
    def __init__(self, quantiles=[0.0025,0.1, 0.5, 0.9, 0.975]):
        self.quantiles = quantiles
        super().__init__() 
    def forward(self, inputs, targets):
        targets = targets.unsqueeze(1).expand_as(inputs)
        quantiles = torch.tensor(self.quantiles).float().to(targets.device)
        error = (targets - inputs).permute(0,2,1)
        loss = torch.max(quantiles*error, (quantiles-1)*error)
        return loss.mean()
    
class BinaryDiceLoss(nn.Module):
    """Dice loss of binary class
    Args:
        smooth: A float number to smooth loss, and avoid NaN error, default: 1
        p: Denominator value: \sum{x^p} + \sum{y^p}, default: 2
        predict: A tensor of shape [N, *]
        target: A tensor of shape same with predict
        reduction: Reduction method to apply, return mean over batch if 'mean',
            return sum if 'sum', return a tensor of shape [N,] if 'none'
    Returns:
        Loss tensor according to arg reduction
    Raise:
        Exception if unexpected reduction
    """
    def __init__(self, smooth=1, p=2, reduction='mean'):
        super(BinaryDiceLoss, self).__init__()
        self.smooth = smooth
        self.p = p
        self.reduction = reduction

    def forward(self, predict, target):
        assert predict.shape[0] == target.shape[0], "predict & target batch size don't match"
        predict = predict.contiguous().view(predict.shape[0], -1)
        target = target.contiguous().view(target.shape[0], -1)

        num = torch.sum(torch.mul(predict, target), dim=1) + self.smooth
        den = torch.sum(predict.pow(self.p) + target.pow(self.p), dim=1) + self.smooth
        loss = 1 - num / den

        if self.reduction == 'mean':
            return loss.mean()
        elif self.reduction == 'sum':
            return loss.sum()
        elif self.reduction == 'none':
            return loss
        else:
            raise Exception('Unexpected reduction {}'.format(self.reduction))
        
class CustomStopper(tf.keras.callbacks.EarlyStopping):
    def __init__(self, monitor='val_loss', min_delta=0.0001, patience=10, verbose=1, mode='auto', start_epoch=5):
        super().__init__(monitor=monitor, min_delta=min_delta, patience=patience, verbose=verbose, mode=mode)
        self.start_epoch = start_epoch

    def on_epoch_end(self, epoch, logs=None):
        print("On epoch End!")
        if epoch > self.start_epoch:
            super().on_epoch_end(epoch, logs)
            print("On epoch End after starting point!")

In [2]:
from sklearn.metrics import mean_squared_error, r2_score

class DatasetForImputation:
    def __init__(self, data, target, mask, seq_len):
        self.data = data
        self.seq_len = seq_len
        self.target = target
        self.mask = mask

    def __len__(self):
        length = len(self.target) - self.seq_len + 1
        if length < 0:
            return 0
        return length

    def __getitem__(self, idx):
        sample = self.data[idx : idx + self.seq_len, 1:]
        target = self.target[idx : idx + self.seq_len]
        mask = self.mask[idx : idx + self.seq_len]
        return sample, target, mask

class Imputation_helper:
    def __init__(self, config):
        self.config = config

    def split_data(self, data, ratio):
        split = int(ratio * len(data))
        train = data[:split]
        test = data[split:]
        return train, test

    def normalize_data(self, data, columns):
        min_max = {}
        mean_std = {}
        for col in columns:
            #min max normalization
            # min_max[col] = [data[col].min(), data[col].max()]
            # data[col] = (data[col] - data[col].min()) / (data[col].max() - data[col].min())

            #standard normalization
            mean_std[col] = [data[col].mean(), data[col].std()]
            data[col] = (data[col] - data[col].mean()) / data[col].std()


        return data, mean_std
    
    def reverse_normalize_data(self, data, columns, min_max):
        for col in columns:
            # data[col] = data[col] * (min_max[col][1] - min_max[col][0]) + min_max[col][0]
            #reverse standard normalization
            data[col] = data[col] * min_max[col][1] + min_max[col][0]
        return data
    
    def get_data_loader(self, data, target, mask, shuffle, ratio = 0.8):
        data = torch.tensor(data, dtype=torch.float32).to(self.config.device)
        target = torch.tensor(target, dtype=torch.float32).to(self.config.device)
        mask = torch.tensor(mask, dtype=torch.float32).to(self.config.device)
        x_train, x_val = self.split_data(data, ratio)
        y_train, y_val = self.split_data(target, ratio)
        mask_train, mask_val = self.split_data(mask, ratio)
        train_dataloader = DataLoader(DatasetForImputation(x_train, y_train, mask_train, self.config.lag_size), batch_size=self.config.batch_size, shuffle=shuffle, drop_last=True)
        val_dataloader = DataLoader(DatasetForImputation(x_val, y_val, mask_val, self.config.lag_size), batch_size=self.config.batch_size, shuffle=shuffle, drop_last=True)
        return train_dataloader, val_dataloader
    
    def convert_to_dataframe(self, data, columns):
        data = pd.DataFrame(data, columns=columns)
        return data
    
    def post_process_data(self, ground_truth, predicted, mask, col):
        ground_truth[col] = (predicted[col] * mask[col]) + (ground_truth[col] * (1 - mask[col]))
        return ground_truth
    
    def evaluation_metrics(self, ground_truth, predicted):
        rmse = np.sqrt(mean_squared_error(ground_truth, predicted))
        mae = np.mean(np.abs(ground_truth - predicted))
        r2 = r2_score(ground_truth, predicted)

        print("RMSE : ", rmse)
        print("MAE : ", mae)
        print("R2 : ", r2)
       
    def replace_data_with_imputed_data(self, data, imputed_data, missing_indexes, col):
        for i in missing_indexes:
            if i not in imputed_data.index:
                continue
            val = imputed_data[imputed_data.index == i][col].values[0]
            data.loc[i, col] = val

        return data

In [3]:
import tensorflow as tf
class ModelEvaluation:
    def __init__(self):
        pass

    def get_memory_stats(self, model, input_shape=(), batch_size=1):
        """Get basic memory statistics for a model"""
        
        # Parameter memory
        total_params = sum(p.numel() for p in model.parameters())
        param_memory_mb = total_params * 4 / (1024**2)  # 4 bytes per float32
        
        # Inference memory (approximate)
        model.eval()
        dummy_input = torch.randn(batch_size, *input_shape)
        
        # Measure peak memory during forward pass
        if torch.cuda.is_available():
            model = model.cuda()
            dummy_input = dummy_input.cuda()
            torch.cuda.empty_cache()
            torch.cuda.reset_peak_memory_stats()
            
            with torch.no_grad():
                _ = model(dummy_input)
            
            inference_memory_mb = torch.cuda.max_memory_allocated() / (1024**2)
        else:
            # For CPU, approximate activation memory
            with torch.no_grad():
                _ = model(dummy_input)
            inference_memory_mb = param_memory_mb * 2  # Rough estimate
        
        return {
            'parameters': total_params,
            'param_memory_mb': param_memory_mb,
            'inference_memory_mb': inference_memory_mb
        }

    def get_tf_memory_stats(self, model, input_shape=(1, 256, 1)):
        """
        Estimate parameter and inference memory usage of a TensorFlow model with input shape
        (batch_size, sequence_length, feature_dim), e.g., for time-series or sequential models.

        Parameters:
            model: tf.keras.Model
            input_shape: tuple like (batch_size, sequence_length, feature_dim)

        Returns:
            dict: parameters, param_memory_mb, inference_memory_mb
        """
        # Total parameters
        
        total_params = model.count_params()
        param_memory_mb = total_params * 4 / (1024 ** 2)  # float32 = 4 bytes

        # Inference memory measurement
        if tf.config.list_physical_devices('GPU'):
            tf.keras.backend.clear_session()
            tf.config.experimental.reset_memory_stats('GPU:0')

            dummy_input = tf.random.normal(input_shape)
            _ = model(dummy_input, training=False)

            mem_info = tf.config.experimental.get_memory_info('GPU:0')
            inference_memory_mb = mem_info['peak'] / (1024 ** 2)  # convert bytes to MB
        else:
            # Estimate for CPU (approximate only)
            dummy_input = tf.random.normal(input_shape)
            _ = model(dummy_input, training=False)
            inference_memory_mb = param_memory_mb * 2  # conservative estimate

        return {
            'parameters': total_params,
            'param_memory_mb': round(param_memory_mb, 2),
            'inference_memory_mb': round(inference_memory_mb, 2)
        }